Imports and functions from task 1 & 2 to be used again in this task. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares as nls

def Read_file(filename):
    data = []
    filename = filename + ".txt"
    file = open(filename, "r")
    for coor in file:
        data.append(coor.split())
    data = np.array(data, dtype='float')
    x = data[:,0]
    y = data[:,1]
    return x, y

def Least_squares(x, y):
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    plt.figure(figsize=(16,14))
    plt.plot(x, y, 'o', label="Original Data", markersize=2)
    plt.plot(x, m*x + c, "r", label="Fitted Line")
    plt.legend()
    plt.show()

loading the data

In [ ]:
x0 = np.array(Read_file("nonlinear_vectorfield_data_x0"))
x1 = np.array(Read_file("nonlinear_vectorfield_data_x1"))
plt.figure(figsize=(16,14))
plt.plot(x0[0,:], x0[1,:], 'o', label="x0", color="blue")
plt.plot(x1[0,:], x1[1,:], 'o', label="x1", color="yellow")
plt.legend()
plt.show()

## 1) Estimate the vector field with a linear operator. 

In [ ]:
dt = 0.1
dphidt = (x1 - x0)/dt

coef = np.linalg.lstsq(dphidt, x0, rcond=None)
A = coef[0]
print('linear operator: A = ')
print(A)

In [ ]:
#estimating x1
x1_hat = x0.dot(A)

plt.figure(figsize=(16,14))
plt.title(r'$x_1$ and $\hat{x_1}$ comparative plots')
plt.plot(x1[0,:], x1[1,:], 'o', label=r"$x_1$", color="blue")
plt.plot(x1_hat[0,:], x1_hat[1,:], 'o', label=r"$\hat{x}_1$", color="red")
plt.legend()
plt.show()

In [ ]:
#computation of the mean square error

def MSE(y,y_pred): 
    mse = 0
    for k in range(y.shape[0]) : 
        error = np.linalg.norm(y-y_pred)**(2)
        mse += error
    return mse


mse = MSE(x1,x1_hat)
print('The mean square error is : ' + str(mse))

## 2) Radial basis function approximation

In [ ]:
def phi(C,x) : 
    ''' 
    The radial function, as the one see in class id est : 
    exp(-||xl - x||**(2)/eps**(2))
    L is the number of coordinates for the vector C, we then choose the x_l, centers of basis functions distributed between
    the maximum and the minimum of the values of x. 
    
    '''
    eps = 0.1
    L = len(C)
        
    x_max = np.amax(x)
    x_min = np.amin(x)
        
        
    x_ls =  np.linspace(x_min,x_max,num = L)
        
        
        
    f_hat = np.zeros(x.shape)
    for l in range(L) : 
        phi_l = np.exp(-np.power(x_ls[l]*np.ones(x.shape)-x,2)/np.linalg.norm(eps)**(2))
        f_hat += phi_l*C[l]
           
        
    return f_hat


def fun(C,x,y) :
    '''The function we want to minimise with the least squares minimisation'''
    return phi(C,x) - y
    
    

    
    
    
def radial_function_solving(x0,x1,dt,L) : 
    '''Given the data points x0 and x1, the time difference and L, the 
    number of variables in the radial function we want, we compute the prediction of 
    the next coordinates'''
    
    dphidt = (x1 - x0)/dt # the speed
    
    x_hat = np.zeros(x0.shape)
    Cs = []
    for i in range(x0.shape[0]) :
        C0 = np.ones(L) #initial guess of c-coefficients
        res_lsq = nls(fun, C0, args=(x0[i,:], dphidt[i,:]))
        C = res_lsq.x

        f_hat = phi(C,x0[i,:])
        x_hat[i,:] = f_hat*dt + x0[i,:]
        Cs.append(C)
    
    return x_hat,Cs


In [ ]:
dt = 0.1
L = 500
x_hat,Cs = radial_function_solving(x0,x1,dt,L)
mse = MSE(x1,x_hat)
print('The mean square error is : ' + str(mse))

In [ ]:
plt.figure(figsize=(16,14))
plt.title(r'$x_1$ and $\hat{x_1}$ comparative plots')
plt.plot(x1[0,:], x1[1,:], 'o', label=r"$x_1$", color="blue")
plt.plot(x_hat[0,:], x_hat[1,:], 'o', label=r"$\hat{x}_1$", color="red")
plt.legend()
plt.show()

## 3) Approximated vector field

a) ploting the approximated vector field

In [ ]:
v = (x_hat - x0)/dt
dx = v[:,0]
dy = v[:,1]

plt.figure(figsize=(16,14))
plt.quiver(x0[:,0], x0[:,1], dx, dy)
plt.show()

b) solving the system for a longer time

In [ ]:
def long_time_solving(x0,x1,L,dt=0.1,num_iter = 20) : 
    states = [x0]
    for i in range(num_iter) : 
        x_hat = radial_function_solving(x0,x1,dt,L)
        states.append(x_hat)
        
        v = (x_hat - x0)/dt
        x0 = x_hat
        x1 = v*dt + x0
        
    return states
        

In [ ]:
states = long_time_solving(x0,x1,L,dt=0.1,num_iter = 20)
